In [1]:
!pip install optuna dill xgboost==1.7.6 catboost lightgbm numpy==1.26.4 pandas scikit-learn

In [2]:
from trainer import *
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split


/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
train_df = pd.read_csv('train.csv').drop(["efs_time"], axis=1)
test_df = pd.read_csv('test.csv')
train_df

,id,store_sales(in millions),unit_sales(in millions),total_children,num_children_at_home,avg_cars_at home(approx).1,gross_weight,recyclable_package,low_fat,units_per_case,store_sqft,coffee_bar,video_store,salad_bar,prepared_food,florist,cost
0,0,8.61,3.0,2.0,2.0,2.0,10.30,1.0,0.0,32.0,36509.0,0.0,0.0,0.0,0.0,0.0,62.09
1,1,5.00,2.0,4.0,0.0,3.0,6.66,1.0,0.0,1.0,28206.0,1.0,0.0,0.0,0.0,0.0,121.80
2,2,14.08,4.0,0.0,0.0,3.0,21.30,1.0,0.0,26.0,21215.0,1.0,0.0,0.0,0.0,0.0,83.51
3,3,4.02,3.0,5.0,0.0,0.0,14.80,0.0,1.0,36.0,21215.0,1.0,0.0,0.0,0.0,0.0,66.78
4,4,2.13,3.0,5.0,0.0,3.0,17.00,1.0,1.0,20.0,27694.0,1.0,1.0,1.0,1.0,1.0,111.51
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
360331,360331,7.60,4.0,5.0,5.0,3.0,13.50,1.0,0.0,33.0,30268.0,0.0,0.0,0.0,0.0,0.0,133.42
360332,360332,14.44,4.0,4.0,0.0,4.0,18.80,1.0,1.0,18.0,20319.0,0.0,0.0,0.0,0.0,0.0,81.85
360333,360333,10.74,3.0,0.0,0.0,2.0,11.30,1.0,0.0,35.0,30584.0,1.0,1.0,1.0,1.0,1.0,87.07
360334,360334,11.04,3.0,1.0,0.0,3.0,10.20,0.0,1.0,14.0,30584.0,1.0,1.0,1.0,1.0,1.0,146.72


In [6]:
train_df, eval_df = train_test_split(train_df, test_size=0.1, random_state=42)
train_df

,id,store_sales(in millions),unit_sales(in millions),total_children,num_children_at_home,avg_cars_at home(approx).1,gross_weight,recyclable_package,low_fat,units_per_case,store_sqft,coffee_bar,video_store,salad_bar,prepared_food,florist,cost
70597,70597,1.74,3.0,5.0,0.0,3.0,17.40,0.0,0.0,32.0,38382.0,0.0,0.0,0.0,0.0,0.0,87.07
259204,259204,4.66,2.0,4.0,0.0,2.0,19.10,0.0,0.0,22.0,30797.0,1.0,1.0,1.0,1.0,1.0,99.38
247515,247515,3.72,4.0,1.0,0.0,3.0,14.00,0.0,0.0,3.0,27694.0,1.0,1.0,1.0,1.0,1.0,131.75
76802,76802,2.43,3.0,1.0,0.0,2.0,6.33,1.0,1.0,20.0,34452.0,1.0,0.0,0.0,0.0,1.0,59.40
92023,92023,4.58,2.0,3.0,3.0,3.0,15.00,0.0,0.0,31.0,27694.0,1.0,1.0,1.0,1.0,1.0,59.86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
119879,119879,9.20,4.0,2.0,0.0,1.0,21.50,1.0,0.0,29.0,23688.0,1.0,1.0,1.0,1.0,1.0,68.84
259178,259178,2.52,2.0,3.0,0.0,1.0,19.80,0.0,1.0,8.0,22478.0,1.0,0.0,0.0,0.0,0.0,119.30
131932,131932,4.56,4.0,2.0,1.0,4.0,14.80,1.0,1.0,18.0,23598.0,0.0,0.0,0.0,0.0,1.0,64.76
146867,146867,10.44,4.0,5.0,0.0,4.0,21.70,1.0,1.0,6.0,38382.0,0.0,0.0,0.0,0.0,0.0,62.74


In [4]:
from sklearn.metrics import mean_squared_log_error

rmsle = Scorer(name="rmsle",
               scorer=mean_squared_log_error,
               greater_is_better=False,
               extra_params={"squared": False}
             )



preprocessor = PreprocessingTool(val_folds = True,
                                 n_folds = 5,
                                 seed = 42,
                                 prob_type="regression",
                                 drop_non_categorical_text=False
                                 )

train_dataset = TabularDataset(train_df,
                               label = "cost",
                               preprocessor = preprocessor,
                               type = "train")
eval_dataset = TabularDataset(eval_df,
                              label = "cost",
                              preprocessor = preprocessor,
                              type = "eval")
test_dataset = TabularDataset(test_df,
                              preprocessor = preprocessor,
                              type = "infer")

trainer = Trainer(train_dataset,
                  eval_dataset = eval_dataset,
                  eval_metric = "rmse",
                  models = [
                            # "LGB", 
                            # "XGB", "CAT", 
                            # "RF",
                            "SGD_LINEAR"
                            ],
                  early_stopping_rounds = 0,
                  of_mitigation_level = 0.2,
                  use_gpu = True,
                  use_cuda = True,
                  n_trials = 1000,
                  timeout = 300,
                  meta_timeout = 100,
                  seed = 42,
                  select_top = 3,
                  train_meta = True)

trainer.train()

LABEL: cost
Problem type: regression. 
-----------------------------------------------------------------------------------------------------------------------------------------
IMPUTER:

Done. 
-----------------------------------------------------------------------------------------------------------------------------------------
DTYPES DETECTOR:

Found 3 unique raw np.dtype(s): [dtype('int64') dtype('O') dtype('float64')].
Dropped 1 column(s) with index ID / non-categorical text features.
Converted 0 column(s) to numeric types.
Remaining features: 0 int feat(s), 15 float feat(s), 0 categorical feat(s).
-----------------------------------------------------------------------------------------------------------------------------------------
5-FOLD STRATIFIEDKFOLDREG TRAIN-VAL SPLITTER:

Done.
-----------------------------------------------------------------------------------------------------------------------------------------
SCALER:

Done.
---------------------------------------------

  0%|          | 0/1000 [00:00<?, ?it/s]

Trial 0 finished with values: {'mean_train_rmsle': 0.30173538989527876, 'mean_val_rmsle': 0.30397285784253053, 'optimized_metric': 0.3044203514319809},
and parameters: {'metric': 'None', 'learning_rate': 0.026093625273383002, 'n_estimators': 940, 'max_depth': 6, 'num_leaves': 34, 'feature_fraction': 0.5179712056139774, 'bagging_fraction': 0.6128955343036653, 'lambda_l1': 0.5065338851509116, 'lambda_l2': 0.08153096861451228, 'min_child_samples': 80, 'boost_from_average': True, 'verbosity': -1}.

Trial 1 finished with values: {'mean_train_rmse': 0.5598011031708087, 'mean_val_rmse': 0.5667542359256589, 'optimized_metric': 0.5681448624766289},
and parameters: {'alpha': 1.929475747073748e-06, 'penalty': 'l1', 'l1_ratio': 0.7187692040615182, 'learning_rate': 'adaptive', 'eta0': 0.011811476722751488, 'power_t': 0.30523241276642055, 'max_iter': 945, 'tol': 2.7091302711408002e-05, 'shuffle': True, 'early_stopping': False, 'n_iter_no_change': 10}.

Trial 2 finished with values: {'mean_train_rmse

id  mean_train_rmsle  mean_val_rmsle  optimized_metric  \
0  LGB_0          0.301735        0.303973           0.30442   

   params_bagging_fraction  params_boost_from_average  \
0                 0.612896                       True   

   params_feature_fraction  params_lambda_l1  params_lambda_l2  \
0                 0.517971          0.506534          0.081531   

   params_learning_rate  ...  params_metric params_min_child_samples  \
0              0.026094  ...           None                       80   

   params_n_estimators  params_num_leaves  params_verbosity  \
0                  940                 34                -1   

           datetime_start          datetime_complete               duration  \
0 2024-11-13 00:57:09.807 2024-11-13 01:02:04.908633 0 days 00:04:55.101633   

      state                                              model  
0  COMPLETE  LGBMRegressor(bagging_fraction=0.6128955343036...  

[1 rows x 21 columns]

------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ 


OPTUNING XGB MODEL...



...

  0%|          | 0/1000 [00:00<?, ?it/s]

Trial 0 finished with values: {'mean_train_rmsle': 0.3037651313580175, 'mean_val_rmsle': 0.30420496554123455, 'optimized_metric': 0.30429293237787797},
and parameters: {'learning_rate': 0.0362941751771155, 'n_estimators': 121, 'max_depth': 9, 'reg_lambda': 0.0014981717769800365, 'random_strength': 0.8477939804878512, 'bagging_temperature': 0.6687307037833348, 'border_count': 231, 'min_data_in_leaf': 38, 'verbose': False}.

Trial 1 finished with values: {'mean_train_rmsle': 0.31514736365155216, 'mean_val_rmsle': 0.31515933181011413, 'optimized_metric': 0.3151617254418265},
and parameters: {'learning_rate': 0.005296837063853847, 'n_estimators': 100, 'max_depth': 5, 'reg_lambda': 0.0023335356361887305, 'random_strength': 0.01862417289934004, 'bagging_temperature': 0.4730246936710585, 'border_count': 82, 'min_data_in_leaf': 25, 'verbose': False}.

Trial 2 finished with values: {'mean_train_rmsle': 0.3038101006760263, 'mean_val_rmsle': 0.3047074564443878, 'optimized_metric': 0.3048869275980

,id,mean_train_rmsle,mean_val_rmsle,optimized_metric,params_bagging_temperature,params_border_count,params_learning_rate,params_max_depth,params_min_data_in_leaf,params_n_estimators,params_random_strength,params_reg_lambda,params_verbose,datetime_start,datetime_complete,duration,state,model
0,CAT_0,0.303765,0.304205,0.304293,0.668731,231,0.036294,9,38,121,0.847794,0.001498,False,2024-11-13 01:03:43.287983,2024-11-13 01:03:59.710478,0 days 00:00:16.422495,COMPLETE,<catboost.core.CatBoostRegressor object at 0x0...
1,CAT_2,0.303810,0.304707,0.304887,0.317580,144,0.076930,4,19,796,0.086270,0.110396,False,2024-11-13 01:04:08.981862,2024-11-13 01:04:58.784133,0 days 00:00:49.802271,COMPLETE,<catboost.core.CatBoostRegressor object at 0x0...
2,CAT_1,0.315147,0.315159,0.315162,0.473025,82,0.005297,5,25,100,0.018624,0.002334,False,2024-11-13 01:03:59.850589,2024-11-13 01:04:08.869445,0 days 00:00:09.018856,COMPLETE,<catboost.core.CatBoostRegressor object at 0x0...


------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------ 


OPTUNING RF MODEL...


  0%|          | 0/1000 [00:00<?, ?it/s]

In [5]:
processed_test_data = test_dataset.process()
processed_test_data

,hla_match_c_high,hla_high_res_8,hla_low_res_6,hla_high_res_6,hla_high_res_10,hla_match_dqb1_high,hla_nmdp_6,hla_match_c_low,hla_match_drb1_low,hla_match_dqb1_low,...,donor_related_Related,donor_related_Unrelated,melphalan_dose_MEL,melphalan_dose_N_A__Mel_not_given,cardiac_No,cardiac_Not_done,cardiac_Yes,pulm_moderate_No,pulm_moderate_Not_done,pulm_moderate_Yes
0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
1,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0
2,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,...,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0


In [6]:
preds = trainer.predict(processed_test_data, mode="meta")
preds

array([0.17022137, 0.55481032, 0.1286355 ])

In [8]:
submission = pd.DataFrame({
    "ID": test_df["ID"],
    "prediction": preds
})
submission

,ID,prediction
0,28800,0.170221
1,28801,0.554810
2,28802,0.128636


In [9]:
submission.to_csv("submission_meta.csv", index=False)